# EP05: 리눅스와 윈도우 컨테이너 같이 쓰기

오프닝: 간단한 본인 소개

Q: 이번에 살펴볼 내용은 어떤 내용인가요?

A: 이번 에피소드에서는 리눅스와 윈도우 컨테이너를 같은 클러스터 안에서 동시에 사용하면서, 서로 다른 운영 체제를 사용하는 애플리케이션을 하나의 쿠버네티스에서 구동하는 방법을 살펴보려고 합니다.

Q: 원래 도커나 쿠버네티스는 리눅스에서만 쓸 수 있지 않나요?

A: 윈도우 서버 2016이 출시된 때에 처음으로 윈도우 컨테이너에 대한 지원이 추가되었습니다. 그리고 최근 나온 윈도우 서버 2019에서는 윈도우 쿠버네티스까지 쓸 수 있게 만들어졌습니다. 그래서 윈도우와 리눅스가 사이좋게 쿠버네티스 안에서 일을 할 수 있게 되었습니다.

Q: 그럼 지금 배포할 애플리케이션에서는 어떻게 두 가지 운영 체제를 동시에 사용하나요?

A: AzureVote 라는 샘플 애플리케이션을 약간 변형해서 배포해보려고 합니다. 원래는 모든 컨테이너가 리눅스에서 구동되지만, 지금 배포할 버전은 백엔드에 해당하는 레디스 컨테이너를 윈도우 버전으로 바꾸어서 실행해보려고 합니다.

이상이 없다면 리눅스 파이썬 컨테이너인 프론트엔드에서는 투표 기능이 제대로 작동하게 됩니다.

Q: 레디스가 윈도우에서 실행되기도 하는군요?

A: 최신 버전은 아니지만, 한 때 윈도우 서버용으로 마이크로소프트 오픈테크라는 자회사에서 레디스를 포함한 몇 가지 유명한 오픈 소스 도구를 직접 윈도우 서버용으로 포팅해서 오픈 소스로 공개했던 적이 있습니다. 이것을 활용해보려고 합니다.

지금부터 윈도우와 리눅스 컨테이너를 모두 지원하는 쿠버네티스 클러스터를 만들고, 샘플 애플리케이션을 배포하는 과정을 살펴보도록 하겠습니다.

## 애저 로그인 상태 확인

우선 애저 커맨드 라인 도구의 로그인 상태를 먼저 확인합니다. 로그인 상태가 아니면 다른 터미널에서 `az login` 명령을 실행합니다. 설치되어있지 않다면 OS 버전 별로 애저 커맨드 라인 도구를 설치하고 준비합니다.

In [ ]:
az account list

## AKS 엔진 설치 상태 확인

아래 명령어를 실행하여 `aks-engine`이 설치되었는지 확인합니다. 설치되어있지 않다면 OS 버전 별로 각각 설치 가이드를 따라 설치를 진행합니다.

In [ ]:
aks-engine version

## 서비스 주체 생성

쿠버네티스 클러스터를 만든 후에는 클러스터 수준에서 스스로 애저 리소스들을 제어할 수 있어야 하므로, 서비스 주체 계정을 만들어 계속 해당 계정을 사용하도록 설정합니다.

In [ ]:
$SubscriptionData = (az account list -o json) | ConvertFrom-Json -Depth 16
$SubscriptionId = $SubscriptionData[0].id
$SvcPrincipal = (az ad sp create-for-rbac `
  -n="AzurePlaylist" --role="Contributor" `
  --scopes="/subscriptions/$SubscriptionId" `
  -o json) | ConvertFrom-Json -Depth 16

$PrincipalId = $SvcPrincipal.AppId
$PrincipalPwd = $SvcPrincipal.Password

"Your App ID is $PrincipalId"

## SSH 키 준비하기

아래 명령어를 실행하여 SSH 키의 공개 키를 준비합니다. 설치되어있지 않다면 SSH 키 쌍을 만듭니다.

In [ ]:
$PublicKeyValue = Get-Content -Path "$HOME\.ssh\id_rsa.pub"

## API 모델 준비하기

이제 API 모델을 준비할 차례입니다. AKS 엔진은 코드 기반 인프라 관리 방식을 구현하기 위하여 JSON API 모델을 작성하면, 여기에 맞추어 ARM 템플릿을 만들어줍니다. 이것을 애저 명령줄 도구를 통해 배포하면 손쉽게 쿠버네티스 클러스터를 생성할 수 있습니다.

In [ ]:
$ApiModel = @"
{
  'apiVersion': 'vlabs',
  'properties': {
    'orchestratorProfile': {
      'orchestratorType': 'Kubernetes'
    },
    'servicePrincipalProfile': {
      'clientId': '',
      'secret': ''
    },
    'masterProfile': {
      'count': 1,
      'dnsPrefix': '',
      'vmSize': 'Standard_D2_v3'
    },
    'agentPoolProfiles': [
      {
        'name': 'linuxpool1',
        'count': 2,
        'vmSize': 'Standard_D2_v3',
        'availabilityProfile': 'AvailabilitySet'
      },
      {
        'name': 'windowspool2',
        'count': 2,
        'vmSize': 'Standard_D2_v3',
        'availabilityProfile': 'AvailabilitySet',
        'osType': 'Windows'
      }
    ],
    'windowsProfile': {
      'adminUsername': 'azureuser',
      'adminPassword': 'qDsXQmcZ57mrGuPz'
    },
    'linuxProfile': {
      'adminUsername': 'azureuser',
      'ssh': {
        'publicKeys': [
          {
            'keyData': ''
          }
        ]
      }
    }
  }
}
"@ | ConvertFrom-Json -Depth 16

## API 모델 내용 변경하기

나중에 리눅스 마스터 노드에 접속할 수 있도록 현재 컴퓨터에서 사용하는 SSH 공개 키를 배포할 때 같이 등록합니다. 그리고 서비스 주체를 가리키는 아이디 값과 고유 비밀 키 값을 API 모델에 같이 지정합니다.

In [ ]:
$ApiModel.properties.linuxProfile.ssh.publicKeys[0].keyData = "$PublicKeyValue"
$ApiModel.properties.servicePrincipalProfile.clientId = "$PrincipalId"
$ApiModel.properties.servicePrincipalProfile.secret = "$PrincipalPwd"

In [ ]:
$ApiModel | ConvertTo-Json -Depth 16 | Out-File -Encoding utf8 -Path 'ep05.json' -Force

dir 'ep05*'

## API 모델을 ARM 템플릿으로 변환하고 배포하기

다음의 명령어를 실행하여 API 모델을 ARM 템플릿으로 변환하고 곧바로 애저에 배포하도록 하겠습니다. 이 명령어는 최초에 한 번만 실행하고, 그 이후에는 `ep03_output` 디렉터리 안에 있는 파일들을 이용하여 클러스터를 관리하도록 합니다.

In [ ]:
$DnsPrefix = 'rkttuep05'
$ResourceGroup = 'rkttuep05'
$Location = 'westus2'

In [ ]:
aks-engine deploy --dns-prefix "$DnsPrefix" `
    --resource-group "$ResourceGroup" `
    --location "$Location" `
    --api-model "ep05.json" `
    --auto-suffix `
    --output-directory "ep05_output" `
    --auth-method cli

## kubectl 실행해보기

클러스터가 잘 만들어졌는지 확인해보기 위하여 다음과 같이 `KUBECONFIG` 환경 변수를 설정하여 클러스터에 접근하고, 노드와 구성 상태를 확인합니다.

In [ ]:
$env:KUBECONFIG="ep05_output/kubeconfig/kubeconfig.$Location.json"
kubectl get nodes -o wide

## 리소스 그룹 정리

테스트를 위해 생성한 리소스 그룹을 제거하고 모든 리소스를 일괄 정리합니다.

In [ ]:
az group delete --name $ResourceGroup --yes --no-wait

클로징: 이 동영상을 보고 애저를 써봐야겠다고 생각하셨나요? 애저 계정은 누구나 무료로 만들 수 있습니다. 지금 바로 시작해보세요. (가능하다면 할인/무료 쿠폰을 포함한 가입 링크 제공) 다음 에피소드에서는 클라우드 셸을 사용할 수 있는 다양한 방법을 알아보겠습니다.